<a href="https://colab.research.google.com/github/nmurillon/DeepLearningChallenge/blob/main/ZZ3-DL-GRAVEGEAL-MURILLON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Imports


In [13]:
import tensorflow as tf
import pandas as pd
import numpy as np
import time
from PIL import Image
import glob
import matplotlib.pyplot as plt
%matplotlib inline

# Ajouter les imports nécessaires
import os #Changer le répertoire courant pour les chemins des fichiers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import ModelCheckpoint

##Variables globales

In [ ]:
num_epochs = 100
batch_size = 250

img_shape = (28,28,1)
img_size = img_shape[:2]

num_classes = 5
stop_freeze = 4


##Données

In [16]:
#On monte le drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#on change le répertoire courant pour faciliter l'accès aux données
os.chdir('/content/drive/MyDrive/DeepLearning/') 

In [ ]:
classes = ["basket","eye","binoculars","rabbit","hand"]

train = pd.read_csv('train.csv')
valid = pd.read_csv('valid.csv')

In [ ]:
plt.figure(figsize=(18,9))
for i in range(0,5):
    ax= plt.subplot(3,2 ,i+1)
    mydata = pd.read_csv("train.csv",skiprows = [1], nrows=1)
    im = Image.open('images/'+classes[i]+'/'+ os.listdir("images/"+classes[i])[0])
    fig=ax.imshow(im)
    plt.title(classes[i])
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
plt.show()    


In [ ]:
# TODO
def Data():
    data_dir = os.getcwd()
    data_dir = pathlib.Path(data_dir)

    image_number = len(list(data_dir.glob('*/*.jpg')))
    print("Nombre d'images :", image_number)
    classes = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"])
    print("Classes : ",classes)

    # Augmentation de données : 75% entraînement, 25% test
    image_gen = ImageDataGenerator(
        rescale = 1./255, 
        # rotation_range=10, # rotation
        # zoom_range=0.2, # zoom
        # horizontal_flip=True, # horizontal flip
        # brightness_range=[0.2,1.2], # brightness
        validation_split=0.25) 
        
    train_data_gen = image_gen.flow_from_directory(data_dir, target_size=image_size, color_mode='rgb', shuffle = True, class_mode='categorical', subset='training')
    # Augmentation de la base de test/validation
    test_data_gen = image_gen.flow_from_directory(data_dir, target_size=image_size, color_mode='rgb', shuffle = True, class_mode='categorical', subset='validation')

    return train_data_gen, test_data_gen, classes

##Modèle 1 : Fine tuning

Dans un premier temps, nous utiliserons le transfer Learning

In [ ]:
def TransferModel(input_shape,stop_freeze, model):
    '''
    @param model is a model from tensorflow.keras.applications
    '''
    pretrained_model = MobileNetV2(input_shape=image_shape)
    
    # On ôte la dernière couche de classification
    pretrained_model.layers.pop()   
    
    inputs = Input(input_shape)
    x = pretrained_model(inputs)
    # On fige tous les poids sauf ceux des stop_freeze dernières couches
    for i in range(len(pretrained_model.layers) - stop_freeze):
      pretrained_model.layers[i].trainable = False
    
    model = Sequential([pretrained_model,
                        Dense(num_classes, activation='softmax')])
    
    return model

##Entraînement du réseau

In [ ]:
#x_train, x_test = Data()
#pretrained_model = # TODO
#pretrained_model_name = ""
model = TransferModel(img_shape, stop_freeze, pretrained_model)

# Callback pour la sauvegarde du meilleur modèle
if not os.path.isdir("sauve"):
    os.mkdir("sauve")
    
checkpoint = ModelCheckpoint(f"sauve/{pretrained_model_name}-loss-{val_loss:.2f}-acc-{accuracy:.2f}.h5",
                                save_best_only=True,verbose=1)


# TODO !!!
train_steps_epoch = np.ceil(x_train.samples / batch_size)
val_steps_epoch = np.ceil(x_test.samples / batch_size)

# Entraînement
model.fit(x_train, steps_per_epoch=train_steps_epoch,
                        validation_data=x_test, validation_steps=val_steps_epoch,
                        epochs=num_epochs, verbose=1, callbacks=[checkpoint])